In [ ]:
# downloading the synthetic contacts datasets
!wget https://dataverse.lib.virginia.edu/api/access/datafile/24096 -O contacts-by-age.tar.gz
!tar -zxf contacts-by-age.tar.gz
!mkdir graph_edge_list
!mkdir ga_graph
!mkdir wy_graph_edge_list
!ls 
!pwd

--2022-11-28 21:10:43--  https://dataverse.lib.virginia.edu/api/access/datafile/24096
Resolving dataverse.lib.virginia.edu (dataverse.lib.virginia.edu)... 128.143.228.183
Connecting to dataverse.lib.virginia.edu (dataverse.lib.virginia.edu)|128.143.228.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2350068 (2.2M) [application/x-gzip]
Saving to: ‘contacts-by-age.tar.gz’

contacts-by-age.tar 100%[===================>]   2.24M  1.15MB/s    in 2.0s    

2022-11-28 21:10:45 (1.15 MB/s) - ‘contacts-by-age.tar.gz’ saved [2350068/2350068]

mkdir: cannot create directory ‘graph_edge_list’: File exists
mkdir: cannot create directory ‘ga_graph’: File exists
Alabama_athome		Kentucky_weekend	Ohio_weekend
Alabama_weekday		Louisiana_athome	Oklahoma_athome
Alabama_weekend		Louisiana_weekday	Oklahoma_weekday
Alaska_athome		Louisiana_weekend	Oklahoma_weekend
Alaska_weekday		Maine_athome		Oregon_athome
Alaska_weekend		Maine_weekday		Oregon_weekday
Arizona_athome		Maine_week

In [ ]:
import os
import networkx as nx
import pandas as pd
import numpy as np
from pathlib import Path
import time
import pickle

# extracting the file names for each county in georgia weekday folder
# target state
ga_dir = 'Georgia_weekday/'
# output folder names
ga_graph_edge = 'graph_edge_list/'
ga_graphs = 'ga_graph/'


# ga_dir = 'Wyoming_weekday/'
# ga_graph_edge = 'wy_graph_edge_list/'
# ga_graphs = 'wy_graph/'

ga_files = os.listdir(ga_dir)
# creatinf file names for graph edge list for each county
ga_GE_files = [ga_graph_edge+Path(f).stem+'.GraphEdge.txt' for f in ga_files]

len(ga_files)
ga_files

['ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13049.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13063.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13231.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13077.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13261.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13223.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13127.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13239.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13037.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13279.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13205.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13031.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13291.txt',
 'ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13141.txt',
 'ga_contact_network_config_min_5_

In [ ]:

def create_sub_graph(n_contacts,group_a,group_b,n_group_a,n_group_b):
  '''
  
  '''
  n_size = n_contacts*2

  a = np.random.randint(low=1,high=n_group_a,size=n_size)
  b = np.random.randint(low=1,high=n_group_b,size=n_size)
  a = np.array([a,b]).T
  df_a = pd.DataFrame(a)

  df_a[0] = group_a + df_a[0].astype(str)
  df_a[1] = group_b + df_a[1].astype(str)
  df_a = df_a[df_a[0]!=df_a[1]]
  df_a = df_a.head(n_contacts)
  # df_a
  return df_a.values


def create_network(f):
  print('started network creation for',f)
  df = pd.read_csv(f)
  base_name = Path(f).stem
  # print(base_name)
  f = ga_graph_edge+base_name+'.GraphEdge.txt'
  # print(f)
  # return 0

  # df = df[['number_contacts']]
  df_n = df[['#age_group_1','number_in_age_group_1']]
  df_n = df_n.drop_duplicates()
  df_n = df_n.set_index(df_n['#age_group_1'])
  total_contacts = df['number_contacts'].sum()
  print('total contacts:',total_contacts)
  # display(df_n)

  # graph_edge = pd.DataFrame(data={0:[],1:[]})
  graph_edge = pd.DataFrame(data={0:[],1:[]}).values

  for x,row in df.iterrows():
    # print(row)
    g_1 = row['#age_group_1']
    g_2 = row['age_group_2']
    num_contacts = row['number_contacts']

    n_g_1 = df_n.at[g_1,'number_in_age_group_1']
    n_g_2 = df_n.at[g_2,'number_in_age_group_1']
    # print(g_1,g_2,num_contacts)
    # print(n_g_1,n_g_2)
    sub_graph_edge = create_sub_graph(
        n_contacts=num_contacts,
        group_a=g_1,
        group_b=g_2,
        n_group_a=n_g_1,
        n_group_b=n_g_2,

    )
    # display(sub_graph_edge.head())

    # graph_edge = graph_edge.append(sub_graph_edge)
    graph_edge = np.append(graph_edge,sub_graph_edge,axis=0,)
    # print(row['#age_group_1'])
  # graph_edge.to_csv(f,index=False,header=False)
  # f = f+'test'
  np.savetxt(f,graph_edge,fmt='%s',delimiter=',')
  print(f'file written to {f}')
  return graph_edge,base_name
  
create_network(ga_dir+'/wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56007.txt')
# create_sub_graph(10,'a','b',5,5)

# !head graph_edge_list/ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13025.GraphEdge.txt


started network creation for Wyoming_weekday//wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56007.txt
total contacts: 649637
file written to wy_graph_edge_list/wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56007.GraphEdge.txt


(array([['p339', 'p228'],
        ['p190', 'p882'],
        ['p552', 'p1036'],
        ...,
        ['g414', 'g625'],
        ['g942', 'g1474'],
        ['g234', 'g1282']], dtype=object),
 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56007')

In [ ]:
%%time
# takes a lot of time for all ga counties (159 counties, about an hour) 
i = 0
ga_files = ga_files[i:]
print(ga_files)
for ga_f in ga_files:
  i +=1
  print(ga_f,i)
  GE, f_base = create_network(ga_dir+ga_f)

  # tried to save the graph but could not due to space limitations
  # G = nx.Graph()
  # G.add_edges_from(GE)
  # # print(G.number_of_nodes())
  # # save graph object to file
  # pickle.dump(G, open(ga_graphs+f_base+'.pickle', 'wb')) 
  # break

['wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56007.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56021.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56003.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56029.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56005.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56001.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56017.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56027.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56023.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56025.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56031.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56035.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56033.txt', 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56013.txt', 'wy_contact_network_config_min_5_max_100_alpha_

In [ ]:
# zip files to download
# !ls -alh wy_graph_edge_list
# !ls -alh ga_graph
# !zip -r /content/GE_2.zip /content/graph_edge_list/
# !zip -r /content/ga_graphs.zip /content/ga_graph/

In [ ]:
!ls -alh graph_edge_list

In [ ]:
from google.colab import files
# files.download("/content/GE.zip")
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# reading and creating networkx netwwork for specific county

def get_network(f):
    
  graph_i = pd.read_csv(f,header=None)

  graph_i = graph_i.values
  graph_i
  G = nx.DiGraph()
  # G = nx.Graph()
  G.add_edges_from(graph_i)
  num_nodes = G.number_of_nodes()
  print(num_nodes)
  return G
# type(graph_i)

# g1 = get_network('/content/ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13025.GraphEdge.txt')
g1 = get_network('/content/graph_edge_list/ga_contact_network_config_min_5_max_100_alpha_400_2-cm-13239.GraphEdge.txt')


1976


In [ ]:
print(nx.info(g1))
h1 = nx.condensation(g1)
print(nx.info(h1))
h1.edges


DiGraph with 1976 nodes and 61736 edges
DiGraph with 1 nodes and 0 edges


OutEdgeView([])

In [ ]:
# reading and creating networkx netwwork for each county
for f_i in ga_GE_files:
  G = get_network(f_i)
  break


17094


In [ ]:
def info_network(f):
  # print('started network creation for',f)
  df = pd.read_csv(f)
  base_name = Path(f).stem
  # print(base_name)
  f = ga_graph_edge+base_name+'.GraphEdge.txt'
  # print(f)
  # return 0

  # df = df[['number_contacts']]
  df_n = df[['#age_group_1','number_in_age_group_1']]
  df_n = df_n.drop_duplicates()
  df_n = df_n.set_index(df_n['#age_group_1'])
  total_contacts = df['number_contacts'].sum()
  total_nodes = df_n['number_in_age_group_1'].sum()
  # print('total contacts:',total_contacts)
  # return df_n
  # return [total_contacts,base_name,base_name.split('-')[-1],total_nodes]
  return [total_contacts,base_name,base_name.split('-')[-1]]




info_network(ga_dir+'/wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56007.txt')



[649637,
 'wy_contact_network_config_min_5_max_100_alpha_400_2-cm-56007',
 '56007',
 15009]

In [ ]:
# creates a table of summary
info = [info_network(ga_dir+ga_f) for ga_f in ga_files]
info = pd.DataFrame(info).sort_values(by=[0],ascending=False)
rename_cols = {
    '0': 'num_contacts',
    '1': 'basename ',
    '2': 'FIPS_code ',
    '3': 'num_nodes',
    
    }

# info = info.rename(rename_cols,axis=1).reindex(columns=rename_cols.values())
# info = info.drop('basename',axis=1)

# info = info.
display(info)

,0,1,2,3
1,4970975,wy_contact_network_config_min_5_max_100_alpha_...,56021,92256
9,4191100,wy_contact_network_config_min_5_max_100_alpha_...,56025,77365
4,2642980,wy_contact_network_config_min_5_max_100_alpha_...,56005,45180
19,2247118,wy_contact_network_config_min_5_max_100_alpha_...,56037,41238
5,2169442,wy_contact_network_config_min_5_max_100_alpha_...,56001,36048
13,1799258,wy_contact_network_config_min_5_max_100_alpha_...,56013,38356
12,1447316,wy_contact_network_config_min_5_max_100_alpha_...,56033,30309
3,1194239,wy_contact_network_config_min_5_max_100_alpha_...,56029,27431
21,1031355,wy_contact_network_config_min_5_max_100_alpha_...,56041,20410
8,861731,wy_contact_network_config_min_5_max_100_alpha_...,56023,18367


In [ ]:
%%time
# only works for georgia
info = [info_network(ga_dir+ga_f) for ga_f in ga_files]
info = pd.DataFrame(info).sort_values(by=[0],ascending=False)
ga_counties = [
    ('13001', 'Appling'),
    ('13003', 'Atkinson'),
    ('13005', 'Bacon'),
    ('13007', 'Baker'),
    ('13009', 'Baldwin'),
    ('13011', 'Banks'),
    ('13013', 'Barrow'),
    ('13015', 'Bartow'),
    ('13017', 'Ben Hill'),
    ('13019', 'Berrien'),
    ('13021', 'Bibb'),
    ('13023', 'Bleckley'),
    ('13025', 'Brantley'),
    ('13027', 'Brooks'),
    ('13029', 'Bryan'),
    ('13031', 'Bulloch'),
    ('13033', 'Burke'),
    ('13035', 'Butts'),
    ('13037', 'Calhoun'),
    ('13039', 'Camden'),
    ('13043', 'Candler'),
    ('13045', 'Carroll'),
    ('13047', 'Catoosa'),
    ('13049', 'Charlton'),
    ('13051', 'Chatham'),
    ('13053', 'Chattahoochee'),
    ('13055', 'Chattooga'),
    ('13057', 'Cherokee'),
    ('13059', 'Clarke'),
    ('13061', 'Clay'),
    ('13063', 'Clayton'),
    ('13065', 'Clinch'),
    ('13067', 'Cobb'),
    ('13069', 'Coffee'),
    ('13071', 'Colquitt'),
    ('13073', 'Columbia'),
    ('13075', 'Cook'),
    ('13077', 'Coweta'),
    ('13079', 'Crawford'),
    ('13081', 'Crisp'),
    ('13083', 'Dade'),
    ('13085', 'Dawson'),
    ('13087', 'Decatur'),
    ('13089', 'De Kalb'),
    ('13091', 'Dodge'),
    ('13093', 'Dooly'),
    ('13095', 'Dougherty'),
    ('13097', 'Douglas'),
    ('13099', 'Early'),
    ('13101', 'Echols'),
    ('13103', 'Effingham'),
    ('13105', 'Elbert'),
    ('13107', 'Emanuel'),
    ('13109', 'Evans'),
    ('13111', 'Fannin'),
    ('13113', 'Fayette'),
    ('13115', 'Floyd'),
    ('13117', 'Forsyth'),
    ('13119', 'Franklin'),
    ('13121', 'Fulton'),
    ('13123', 'Gilmer'),
    ('13125', 'Glascock'),
    ('13127', 'Glynn'),
    ('13129', 'Gordon'),
    ('13131', 'Grady'),
    ('13133', 'Greene'),
    ('13135', 'Gwinnett'),
    ('13137', 'Habersham'),
    ('13139', 'Hall'),
    ('13141', 'Hancock'),
    ('13143', 'Haralson'),
    ('13145', 'Harris'),
    ('13147', 'Hart'),
    ('13149', 'Heard'),
    ('13151', 'Henry'),
    ('13153', 'Houston'),
    ('13155', 'Irwin'),
    ('13157', 'Jackson'),
    ('13159', 'Jasper'),
    ('13161', 'Jeff Davis'),
    ('13163', 'Jefferson'),
    ('13165', 'Jenkins'),
    ('13167', 'Johnson'),
    ('13169', 'Jones'),
    ('13171', 'Lamar'),
    ('13173', 'Lanier'),
    ('13175', 'Laurens'),
    ('13177', 'Lee'),
    ('13179', 'Liberty'),
    ('13181', 'Lincoln'),
    ('13183', 'Long'),
    ('13185', 'Lowndes'),
    ('13187', 'Lumpkin'),
    ('13189', 'McDuffie'),
    ('13191', 'McIntosh'),
    ('13193', 'Macon'),
    ('13195', 'Madison'),
    ('13197', 'Marion'),
    ('13199', 'Meriwether'),
    ('13201', 'Miller'),
    ('13205', 'Mitchell'),
    ('13207', 'Monroe'),
    ('13209', 'Montgomery'),
    ('13211', 'Morgan'),
    ('13213', 'Murray'),
    ('13215', 'Muscogee'),
    ('13217', 'Newton'),
    ('13219', 'Oconee'),
    ('13221', 'Oglethorpe'),
    ('13223', 'Paulding'),
    ('13225', 'Peach'),
    ('13227', 'Pickens'),
    ('13229', 'Pierce'),
    ('13231', 'Pike'),
    ('13233', 'Polk'),
    ('13235', 'Pulaski'),
    ('13237', 'Putnam'),
    ('13239', 'Quitman'),
    ('13241', 'Rabun'),
    ('13243', 'Randolph'),
    ('13245', 'Richmond'),
    ('13247', 'Rockdale'),
    ('13249', 'Schley'),
    ('13251', 'Screven'),
    ('13253', 'Seminole'),
    ('13255', 'Spalding'),
    ('13257', 'Stephens'),
    ('13259', 'Stewart'),
    ('13261', 'Sumter'),
    ('13263', 'Talbot'),
    ('13265', 'Taliaferro'),
    ('13267', 'Tattnall'),
    ('13269', 'Taylor'),
    ('13271', 'Telfair'),
    ('13273', 'Terrell'),
    ('13275', 'Thomas'),
    ('13277', 'Tift'),
    ('13279', 'Toombs'),
    ('13281', 'Towns'),
    ('13283', 'Treutlen'),
    ('13285', 'Troup'),
    ('13287', 'Turner'),
    ('13289', 'Twiggs'),
    ('13291', 'Union'),
    ('13293', 'Upson'),
    ('13295', 'Walker'),
    ('13297', 'Walton'),
    ('13299', 'Ware'),
    ('13301', 'Warren'),
    ('13303', 'Washington'),
    ('13305', 'Wayne'),
    ('13307', 'Webster'),
    ('13309', 'Wheeler'),
    ('13311', 'White'),
    ('13313', 'Whitfield'),
    ('13315', 'Wilcox'),
    ('13317', 'Wilkes'),
    ('13319', 'Wilkinson'),
    ('13321', 'Worth'),
    ]

ga_counties = pd.DataFrame(ga_counties)
# ga_counties[0] = ga_counties[0].astype(int)
info = pd.merge(info, ga_counties, left_on=2, right_on=0)
info = info.drop(2,axis=1)
rename_cols = {
    '1_y': 'county_name',
    '0_y': 'FIPS_code',
    '0_x': 'num_contacts',
    '1_x': 'basename',
    
    }

info = info.rename(rename_cols,axis=1).reindex(columns=rename_cols.values())
info = info.drop('basename',axis=1)

# info = info.
display(info)
print(list(info['county_name']))


,county_name,FIPS_code,num_contacts
0,Fulton,13121,53109221
1,Gwinnett,13135,53019972
2,Cobb,13067,44319093
3,De Kalb,13089,39757734
4,Clayton,13063,15127770
...,...,...,...
154,Glascock,13125,125280
155,Webster,13307,119522
156,Clay,13061,91280
157,Taliaferro,13265,63923


['Fulton', 'Gwinnett', 'Cobb', 'De Kalb', 'Clayton', 'Chatham', 'Cherokee', 'Forsyth', 'Henry', 'Hall', 'Paulding', 'Richmond', 'Muscogee', 'Houston', 'Coweta', 'Douglas', 'Columbia', 'Bibb', 'Clarke', 'Carroll', 'Fayette', 'Newton', 'Whitfield', 'Bartow', 'Lowndes', 'Floyd', 'Walton', 'Rockdale', 'Dougherty', 'Barrow', 'Bulloch', 'Glynn', 'Jackson', 'Catoosa', 'Walker', 'Troup', 'Liberty', 'Effingham', 'Spalding', 'Gordon', 'Camden', 'Oconee', 'Thomas', 'Colquitt', 'Murray', 'Polk', 'Laurens', 'Habersham', 'Bryan', 'Baldwin', 'Tift', 'Harris', 'Coffee', 'Ware', 'Haralson', 'Madison', 'Lee', 'Sumter', 'Jones', 'Lumpkin', 'Toombs', 'Pickens', 'Wayne', 'Gilmer', 'Peach', 'White', 'Decatur', 'Monroe', 'Burke', 'Stephens', 'Grady', 'Hart', 'Upson', 'Chattooga', 'Franklin', 'Tattnall', 'Butts', 'Emanuel', 'McDuffie', 'Dawson', 'Crisp', 'Washington', 'Mitchell', 'Meriwether', 'Putnam', 'Worth', 'Berrien', 'Fannin', 'Pierce', 'Morgan', 'Dodge', 'Banks', 'Long', 'Brantley', 'Elbert', 'Pike', '